In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.2 MB/s 
     |████████████████████████████████| 5.8 MB 51.8 MB/s 
     |████████████████████████████████| 132 kB 43.6 MB/s 
     |████████████████████████████████| 212 kB 25.8 MB/s 
     |████████████████████████████████| 182 kB 42.2 MB/s 
     |████████████████████████████████| 127 kB 7.1 MB/s 
     |████████████████████████████████| 7.6 MB 32.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/ece1786-project/validation

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ece1786-project/validation


In [ ]:
import transformers
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd
import torch 
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
import evaluate
import numpy as np

In [ ]:
model_checkpoint = "gpt2" #"distilgpt2"
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
filename = "drink_val.pkl"
filename1 = filename
filename2 = "cake_val.pkl"
combined = pd.concat([pd.read_pickle(filename1), pd.read_pickle(filename2)], ignore_index=True)
device = torch.device('cpu')

In [ ]:
print(combined.to_markdown())

|    | Name                          | Ingredients                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | Instructions                                                                                                                                                                                                                                                                                                                            

In [ ]:
# def create_propmts(filename, index):
#     df = pd.read_pickle(filename)

#     recipe = "RECIPE NAME\n" + df.loc[index, "Name"] + " \n\nRECIPE INGREDIENTS\n"
#     prompts = [recipe]
#     for ingredient in df.loc[index, "Ingredients"]:
#       recipe += ingredient + "\n"
#     recipe += "\nRECIPE INSTRUCTIONS\n" 
#     for instruction in  df.loc[index, "Instructions"]:
#       recipe += instruction + "\n"
#     answers = [[recipe]]
#     return answers, prompts
def create_propmts(df, index):
    
    recipe = "RECIPE NAME\n" + df.loc[index, "Name"] + " \n\nRECIPE INGREDIENTS\n"
    prompts = [recipe]
    for ingredient in df.loc[index, "Ingredients"]:
      recipe += ingredient + "\n"
    recipe += "\nRECIPE INSTRUCTIONS\n" 
    for instruction in  df.loc[index, "Instructions"]:
      recipe += instruction + "\n"
    answers = [[recipe]]
    return answers, prompts

In [ ]:
# def score(metric, filename, index):
#   answers, prompts = create_propmts(filename, index)
#   attempts = []
#   encoded_prompt = tokenizer(prompts, return_tensors="pt").input_ids.to(device)
#   generated_sequence = model.generate(encoded_prompt, do_sample=True, max_length=len(answers[0][0].split()), temperature = 0.5, return_dict_in_generate=True, output_scores=True)
#   decoded_sequence = tokenizer.batch_decode(generated_sequence.sequences, skip_special_tokens=True)[0]
#   attempts.append(decoded_sequence)
  
#   eval = evaluate.load(metric)
#   results = eval.compute(predictions=attempts, references=answers)
#   return results, decoded_sequence

def score(metric, df, index):
  answers, prompts = create_propmts(df, index)
  attempts = []
  encoded_prompt = tokenizer(prompts, return_tensors="pt").input_ids.to(device)
  generated_sequence = model.generate(encoded_prompt, do_sample=True, max_length=len(answers[0][0].split()), temperature = 0.5, return_dict_in_generate=True, output_scores=True)
  decoded_sequence = tokenizer.batch_decode(generated_sequence.sequences, skip_special_tokens=True)[0]
  attempts.append(decoded_sequence)
  
  eval = evaluate.load(metric)
  results = eval.compute(predictions=attempts, references=answers)
  return results, decoded_sequence

In [ ]:
model_checkpoint = "../gpt2_combined"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

In [ ]:
iterations = 5
cardinality = 10

answers = []
scores = np.zeros((cardinality, iterations))
for i in range(cardinality):
  tmp = []
  for j in range(iterations):
    eins, zwei = score("bleu", combined, i)
    # eins, zwei = score("bleu", filename, i)
    scores[i][j] = eins["bleu"]
    tmp.append(zwei)
  answers.append(tmp[np.argmax(scores[i])])
# answers[1]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [ ]:
with open('combined_val_results.txt', 'w') as fp:
    for item in answers:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
np.mean(scores)

0.07287151577014644

In [ ]:
# prompt = "RECIPE NAME\n"
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cpu")

# outputs = model.generate(input_ids, do_sample=True, max_length=600,temperature = 0.5, top_p = 1)
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

In [ ]:
# df = pd.read_pickle("cake_val.pkl")  
# recipes = []
# for i in range(len(df.index)):
#     recipe = "RECIPE NAME\n" + df.loc[i, "Name"] + " \n\nRECIPE INGREDIENTS\n"
#     skip = False
#     for ingredient in df.loc[i, "Ingredients"]:
#         recipe += ingredient + "\n"
#     if skip:
#         continue
#     recipe += "\nRECIPE INSTRUCTIONS\n" 
#     for instruction in  df.loc[i, "Instructions"]:
#         if instruction.startswith("*"):
#             continue
#         recipe += instruction + "\n"
#     recipes.append(recipe)
# tokenizer.pad_token = tokenizer.eos_token
# data = tokenizer(recipes, padding='longest', truncation=True)
# data["labels"] = data["input_ids"].copy()

# # dataset = Dataset.from_dict(data)
# def create_propmts(filename, index):
#     df = pd.read_pickle(filename)
#     answers = []
#     prompts = []
#     for i in range(len(df.index)):
#       recipe = "RECIPE NAME\n" + df.loc[i, "Name"] + " \n\nRECIPE INGREDIENTS\n"
#       prompts.append(recipe)
#       for ingredient in df.loc[i, "Ingredients"]:
#         recipe += ingredient + "\n"
#       recipe += "\nRECIPE INSTRUCTIONS\n" 
#       for instruction in  df.loc[i, "Instructions"]:
#         recipe += instruction + "\n"
#       answers.append([recipe])
#     return answers, prompts
# def score(metric="bleu", filename="cocktail_dataset.pkl"):
#   answers, prompts = create_propmts()
#   attempts = []
#   for i in range(len(answers)):
#     encoded_prompt = tokenizer(prompts[i], return_tensors="pt").input_ids.to(device)
#     generated_sequence = model.generate(encoded_prompt, do_sample=True, max_length=len(answers[i][0].split()), temperature = 0.5, return_dict_in_generate=True, output_scores=True)
#     attempts.append(tokenizer.batch_decode(generated_sequence.sequences, skip_special_tokens=True)[0])
  
#   eval = evaluate.load(metric)
#   results = eval.compute(predictions=attempts, references=answers)
#   return results